In [ ]:
import pandas as pd
import re
import os

# Function to process each sheet
def process_sheet(sheet):
    # Find the header row
    header_row = None
    for index, row in sheet.iterrows():
        if ('Entity' in row.values) and ('RTN Level 4' in row.values) and ('Account' in row.values):
            header_row = index
            break
    
    if header_row is None:
        print("Header not found in the sheet.")
        return

    # Row above header
    year_row = header_row - 1

    # Identify the Mxx, FY, and YTDxx columns and concatenate them with the row above
    for col in sheet.columns:
        if re.match(r'M\d{2}', col) or col.startswith('FY') or re.match(r'YTD\d{2}', col):
            # Concatenate current cell with the one above
            for idx in range(header_row + 1, len(sheet)):
                sheet.at[idx, col] = f"{sheet.at[year_row, col]} {sheet.at[idx, col]}"

    # Display for testing
    print(f"Processed sheet from header row {header_row}.")

# Main processing function
def main():
    # Get current year and month from user
    current_year = input("Enter current year (e.g., 2024): ")
    current_month = input("Enter current month (1-12): ")

    # Define file paths
    excel_file = 'your_file.xlsx'  # Replace with your file path
    output_file = 'processed_file.xlsx'

    # Check if the output file exists; if so, delete it
    if os.path.exists(output_file):
        os.remove(output_file)
        print(f"Deleted existing file: {output_file}")
    else:
        print(f"No existing file found. Creating a new one: {output_file}")

    # Load the workbook to process
    xls = pd.ExcelFile(excel_file)
    
    # Process each sheet and write to a new workbook
    with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
        for sheet_name in xls.sheet_names:
            if 'excl' not in sheet_name:
                sheet = pd.read_excel(xls, sheet_name=sheet_name)
                process_sheet(sheet)

                # Save processed sheet to the new workbook
                sheet.to_excel(writer, sheet_name=sheet_name, index=False)

    print(f"Workbook saved as: {output_file}")

if __name__ == "__main__":
    main()
